In [ ]:
!pip install transformers torch

In [ ]:
pip install -U bitsandbytes

In [ ]:
import os
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from google.colab import userdata


try:
    hf_token = userdata.get('hf_token')
    if not hf_token:
        print("Warning: HF_TOKEN not found in Colab Secrets under 'hf_token'. Proceeding for public model access.")
        hf_token = None
except Exception as e:
    print(f"Error retrieving HF_TOKEN: {str(e)}")
    print("Warning: Token retrieval failed, but proceeding for public model access.")
    hf_token = None


!pip install -U transformers accelerate -q



hf_token = os.getenv("HF_TOKEN", None)


def generate_questions(intern_profile, job_description):

    prompt = f"""
    You are an expert interview question generator for intern roles. Based on the provided intern profile and job description, generate 5 technical and 5 behavioral interview questions tailored to the candidate's skills and the role's requirements. Ensure technical questions match the candidate’s experience level and the job’s technical needs, and behavioral questions follow the STAR framework (Situation, Task, Action, Result).

    Intern Profile:
    {intern_profile}

    Job Description:
    {job_description}

    Output Format:
    Technical Questions:
    1. [Question]
    2. [Question]
    3. [Question]
    4. [Question]
    5. [Question]

    Behavioral Questions:
    1. [Question]
    2. [Question]
    3. [Question]
    4. [Question]
    5. [Question]
    """
    messages = [
        {"role": "user", "content": prompt}
    ]
    try:

        inputs = tokenizer.apply_chat_template(
            messages,
            add_generation_prompt=True,
            tokenize=True,
            return_dict=True,
            return_tensors="pt"
        ).to(model.device)


        outputs = model.generate(
            **inputs,
            max_new_tokens=600,
            temperature=0.7,
            do_sample=True,
            top_p=0.9
        )

        generated_text = tokenizer.decode(outputs[0][inputs["input_ids"].shape[-1]:], skip_special_tokens=True)
        return generated_text.strip()
    except Exception as e:
        return f"Error generating questions: {str(e)}"


intern_profile_1 = """
- Role: Data Science Intern
- Skills: Python, SQL, basic machine learning
- Experience: Beginner, 2nd-year BSc Computer Science
- Projects: Built a linear regression model for a university project
"""
job_description_1 = """
- Role: Data Science Intern
- Skills Required: Python, data analysis, team collaboration
- Responsibilities: Analyze datasets, build predictive models, present findings
"""

# Sample Input 2: Software Engineering Intern
intern_profile_2 = """
- Role: Software Engineering Intern
- Skills: Python, Java, basic understanding of web development (HTML, CSS)
- Experience: Beginner, 3rd-year BSc Computer Science
- Projects: Developed a simple web application using Flask and Python for a course project
"""
job_description_2 = """
- Role: Software Engineering Intern
- Skills Required: Python, Java, web development, problem-solving
- Responsibilities: Assist in developing backend features, write clean code, participate in code reviews
"""


intern_profile_3 = """
- Role: Product Management Intern
- Skills: Basic Python, user research, communication
- Experience: Beginner, 2nd-year BA Business Administration
- Projects: Conducted user interviews for a mobile app prototype in a class project
"""
job_description_3 = """
- Role: Product Management Intern
- Skills Required: User research, data analysis, teamwork
- Responsibilities: Gather user feedback, assist in defining product requirements, analyze usage data
"""


sample_inputs = [
    ("Data Science Intern", intern_profile_1, job_description_1),
    ("Software Engineering Intern", intern_profile_2, job_description_2),
    ("Product Management Intern", intern_profile_3, job_description_3)
]

for role, intern_profile, job_description in sample_inputs:
    print(f"\n=== {role} Questions ===")
    questions = generate_questions(intern_profile, job_description)
    print(questions)